In [1]:
import psycopg2
import ast
import json
import os
import tqdm

from utils import get_db_params

db_params = get_db_params()
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

In [21]:
# Get 10 random examples of recipes with ingredients 
query = "SELECT R.Name AS RecipeName, string_agg(I.Name, ', ') AS Ingredients \
FROM Recipes R \
JOIN RecipeIngredients RI ON R.ID = RI.RecipeID \
JOIN Ingredients I ON RI.IngredientID = I.IngredientID \
GROUP BY R.ID, R.Name \
ORDER BY RANDOM() \
LIMIT 10;"

cursor.execute(query)
rows = cursor.fetchall()
print("10 random recipes with ingredients:")
for row in rows:
    print(row)
    print("\n")


10 random recipes with ingredients:
(None, 'olive oil, grated Parmesan cheese, ground black pepper, garlic, minced, dried thyme, shredded Cheddar cheese, shredded mozzarella cheese')


(None, 'baking powder, vanilla extract, eggs, butter, white sugar, baking soda')


(None, 'ketchup, salt, water, brown sugar, lemon juice, chopped onion, chopped green bell pepper, mustard powder, celery seed')


(None, "salt, vanilla extract, heavy whipping cream, confectioners' sugar")


(None, "sel, cannelle moulue, eau, oeufs, farine, huile d'olive, gousses d'ail, emincees, pois surgeles")


(None, 'Kosher Salt, Eggs, Olive Oil')


(None, "sel, farine, huile d'olive, sel et poivre noir moulu selon le gout")


(None, 'baking powder, salt, vanilla extract, eggs, flour, butter, white sugar')


(None, 'salt, water, white sugar, white vinegar')


(None, 'vegetable oil, baking powder, salt, eggs, white sugar, baking soda')




In [14]:
query = "SELECT R.Name AS RecipeName, string_agg(I.Name, ', ') AS IngredientNames\
        FROM Recipes R\
        JOIN RecipeIngredients RI ON R.ID = RI.RecipeID\
        JOIN Ingredients I ON RI.IngredientID = I.IngredientID\
        WHERE R.ID IN (\
            SELECT RecipeID\
            FROM RecipeIngredients RI\
            JOIN Ingredients I ON RI.IngredientID = I.IngredientID\
            WHERE I.Name IN ('butter')\
            GROUP BY RecipeID\
        )\
        GROUP BY R.Name;"

cursor.execute("ROLLBACK;")
cursor.execute(query)

rows = cursor.fetchall()
print(len(rows))
for row in rows:
    print(row)
    print()


1
(None, "baking powder, vanilla extract, eggs, flour, sugar, butter, poppy seeds, icing sugar, eggs, sugar, butter, lemons, eggs, sugar, butter, water, double cream, vegetable oil, baking powder, salt, eggs, butter, white sugar, baking soda, baking powder, eggs, sugar, butter, vanilla, milk, powdered sugar, sugar, butter, kosher salt, cold water, sugar, butter, double cream, milk, butter, kosher salt, freshly ground black pepper, whole milk, parsley, eggs, butter, kosher salt, freshly ground black pepper, whole milk, parsley, flour, sugar, butter, sea salt, bicarbonate of soda, eggs, sugar, butter, water, double cream, vegetable oil, baking powder, salt, eggs, butter, white sugar, baking soda, baking powder, eggs, sugar, butter, vanilla, milk, powdered sugar, sugar, butter, kosher salt, cold water, sugar, butter, double cream, milk, butter, kosher salt, freshly ground black pepper, whole milk, parsley, eggs, butter, kosher salt, freshly ground black pepper, whole milk, parsley, eggs, 